In [37]:
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt
import random
import os
import torch
import numpy as np

In [ ]:
pred_len = 24 * 63

In [ ]:
sub = pd.read_csv("./.csv")

In [ ]:
sub["Time"] = pd.to_datetime(sub["Time"])

In [ ]:
class ProphetAVG:
    def __init__(self, scale1=0.01, scale2=0.1):
        self.models = [
            Prophet(seasonality_mode="additive", changepoint_range=1,
                    changepoint_prior_scale=scale1),
            Prophet(seasonality_mode="additive", changepoint_range=1,
                    changepoint_prior_scale=scale2)
        ]

        self.forecasts = []
        self.df = None

    def fit(self, data):
        for model in self.models:
            model.fit(data)

    def predict(self, periods=pred_len, freq="h"):
        future_frames = [model.make_future_dataframe(periods=periods, freq=freq) for model in self.models]
        forecasts = [model.predict(future) for model, future in zip(self.models, future_frames)]
        
        # 두 모델의 평균 예측 생성
        avg_forecast = pd.concat([forecast['yhat'] for forecast in forecasts], axis=1).mean(axis=1)

        self.df = pd.DataFrame({
            "Time": sub["Time"],
            "Close": avg_forecast[-periods:].reset_index(drop=True)
        })

        return self.df
    
    def plot(self):
        plt.figure(figsize=(12, 6))
        plt.plot(self.df["Time"], self.df["Close"], label="Prediction", marker="o", linestyle="-")
        plt.xlabel("Time")
        plt.ylabel("Close Price")
        plt.legend()
        plt.grid(True)
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()